In [1]:
import os
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"E:\Practice\ML-in-production-clone\service_acc\service_acc.json"

In [2]:
client = bigquery.Client()
client

#### Basic Query Process

In [13]:
query = """
    SELECT name, SUM(number) as total_people
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE state = 'TX'
    GROUP BY name, state
    ORDER BY total_people DESC
    LIMIT 5
"""

import time
start_time = time.time()
query_job = client.query(query)
print(time.time() - start_time)
print(type(query_job))
print(query_job)

0.8949966430664062
<class 'google.cloud.bigquery.job.query.QueryJob'>
QueryJob<project=sktblog, location=US, id=37d23446-fa01-4001-a74d-15983f938562>


In [15]:
print("Top 5 USA Names:\n")
for row in query_job:
    print("name={}, count={}".format(row[0], row["total_people"]))

Top 5 USA Names:

name=James, count=272793
name=John, count=235139
name=Michael, count=225320
name=Robert, count=220399
name=David, count=219028


In [ ]:
!pip install db-dtypes

In [16]:
df = query_job.to_dataframe()
df.head()

,name,total_people
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


#### Running parameterized queries
https://cloud.google.com/bigquery/docs/parameterized-queries

In [27]:
sql = """
    SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = @state
    LIMIT @limit
"""

query_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter('state', 'STRING', 'TX'),
        bigquery.ScalarQueryParameter('limit', 'INTEGER', 100)
    ]
)

query_job = client.query(query, job_config=query_config)  # Make an API request.

for row in query_job:
    print("{}: \t{}".format(row.name, row.total_people))

James: 	272793
John: 	235139
Michael: 	225320
Robert: 	220399
David: 	219028


In [28]:
# -----------  To use positional parameters ----------- #
sql = """
    SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = ?
    LIMIT ?
"""

query_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter(None, 'STRING', 'TX'),
        bigquery.ScalarQueryParameter(None, 'INTEGER', 100)
    ]
)

query_job = client.query(query, job_config=query_config)  # Make an API request.

for row in query_job:
    print("{}: \t{}".format(row.name, row.total_people))

James: 	272793
John: 	235139
Michael: 	225320
Robert: 	220399
David: 	219028


In [29]:
# ----------- Using arrays in parameterized querie ----------- #
query = """
    SELECT name, sum(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE gender = @gender
    AND state IN UNNEST(@states)
    GROUP BY name
    ORDER BY count DESC
    LIMIT 10;
"""
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("gender", "STRING", "M"),
        bigquery.ArrayQueryParameter("states", "STRING", ["WA", "WI", "WV", "WY"]),
    ]
)
query_job = client.query(query, job_config=job_config)  # Make an API request.

for row in query_job:
    print("{}: \t{}".format(row.name, row.count))

James: 	238155
Robert: 	235458
John: 	217668
Michael: 	204942
David: 	187928
William: 	169083
Richard: 	134337
Thomas: 	114726
Charles: 	103021
Daniel: 	97706


#### Load Dataframe to BigQuery Table

In [ ]:
!pip install pyarrow

In [ ]:
import pandas

df = pandas.DataFrame(
    {
        'my_string': ['a', 'b', 'c'],
        'my_int64': [1, 2, 3],
        'my_float64': [4.0, 5.0, 6.0],
        'my_timestamp': [
            pandas.Timestamp("1998-09-04T16:03:14"),
            pandas.Timestamp("2010-09-13T12:03:45"),
            pandas.Timestamp("2015-10-02T16:00:00")
        ],
    }
)

In [ ]:
table_id = 'sktblog.ths_test_big_dataset.ths_test_big_tb'
# Since string columns use the "object" dtype, pass in a (partial) schema
# to ensure the correct BigQuery data type.
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("my_string", "STRING"),
])

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)